# PROJEKTS: Knowledge Hub



Situācija
Tu esi tikko pieņemts darbā par MI Inženieri. Uzņēmuma vadītājs tev iedevis mapi UNSORTED_DATA, kurā ir haoss - sapulču ieraksti, tāfeles fotogrāfijas un līgumu melnraksti.

Nevienam nav laika to visu lasīt vai klausīties.

## Tavs Uzdevums:

Uzbūvēt Multimodālu RAG Sistēmu, kas "apēd" šos failus un ļauj direktoram čatā uzdot jautājumus (piemēram: "Cik mēs esam parādā?" vai "Ko nolēma par mārketingu?"), saņemot precīzas atbildes ar atsaucēm.

## Vides sagatavošana

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Instalējam nepieciešamās bibliotēkas
%pip install openai langchain langchain-community langchain-openai chromadb pypdf pymupdf openai-whisper tiktoken
!apt-get install ffmpeg -y  # Nepieciešams priekš Whisper audio apstrādes


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [3]:
import os
import getpass
from pathlib import Path
from google.colab import drive

# 1. Pievienojam Google Drive (ja vēl nav pievienots)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Definējam darba mapi (ceļš no tava ekrānšāviņa)
BASE_PATH = Path("/content/drive/MyDrive/Colab Notebooks/BDA_M1/Lab26_KnowledgeHub")

# 3. Ievadi OpenAI API atslēgu
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ievadi OpenAI API Key: ")

# 4. Pārbaudām failus
required_files = ["project.pdf", "meeting.mp3", "whiteboard.png"] # Pamanīju, ka tev ir .png
missing_files = [f for f in required_files if not (BASE_PATH / f).exists()]

if missing_files:
    print(f"❌ TRŪKST FAILU mapē {BASE_PATH}: {missing_files}")
    print("Pārbaudi failu nosaukumus!")
else:
    print(f"✅ Visi faili atrasti mapē: {BASE_PATH}")
    # Iestatām pilnos ceļus ērtākai lietošanai
    pdf_path = str(BASE_PATH / "project.pdf")
    audio_path = str(BASE_PATH / "meeting.mp3")
    image_path = str(BASE_PATH / "whiteboard.png")

✅ Visi faili atrasti mapē: /content/drive/MyDrive/Colab Notebooks/BDA_M1/Lab26_KnowledgeHub


## Izstrādes soļi (Ingestion & Vector Store)
Šis kods "apēd" failus no Google Drive mapes.
* Whisper lasa audio no Drive ceļa.
* GPT-4o lasa attēlu no Drive ceļa.
* PyPDFLoader lasa PDF no Drive ceļa.

In [4]:
import whisper
from langchain_community.document_loaders import PyPDFLoader
try:
    from langchain_community.document_loaders import PyMuPDFLoader
except Exception:
    PyMuPDFLoader = None
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from openai import OpenAI
import base64
import logging

# Iniciējam OpenAI klientu attēlu apstrādei
client = OpenAI()

print("🔎 Sāku datu apstrādi... (Tas var aizņemt pāris minūtes)")
all_documents = []

# --- A. DATU "APĒŠANA" (Ingestion) ---

# 1. AUDIO APSTRĀDE (Whisper)
print(f"   🎤 Apstrādāju audio: {os.path.basename(audio_path)}...")
whisper_model = whisper.load_model("base")
# Whisper ņem faila ceļu tieši no Drive
result = whisper_model.transcribe(
    audio_path,
    task="transcribe",
    language="en",
    fp16=False,
    condition_on_previous_text=True,
)
segments = result.get("segments", [])
audio_text = "\n".join([f"[{s['start']:.1f}-{s['end']:.1f}] {s['text'].strip()}" for s in segments]) if segments else result.get("text", "")
all_documents.append(Document(page_content=audio_text, metadata={"source": "meeting.mp3"}))


# 2. ATTĒLA APSTRĀDE (GPT-4o Vision)
print(f"   🖼️ Apstrādāju attēlu: {os.path.basename(image_path)}...")

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Kodējam attēlu no Drive
base64_image = encode_image(image_path)

response = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Extract all visible text exactly from this image. If there is a table, transcribe each row. Keep every number exactly as written (including commas, currency symbols, and dates). Then provide a short section named NUMBERS_FOUND that lists all numbers you see.",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{base64_image}", "detail": "high"},
                },
            ],
        }
    ],
    max_tokens=1600,
)
image_description = response.choices[0].message.content
# Add retrieval keywords so numeric queries match this chunk more reliably.
image_text_for_index = (
    "Keywords: numbers, digits, amounts, budget, costs, estimated cost, actual, total budget, current spend.\n\n"
    f"{image_description}"
)
all_documents.append(Document(page_content=image_text_for_index, metadata={"source": "whiteboard.png"}))


# 3. PDF APSTRĀDE
print(f"   📄 Apstrādāju PDF: {os.path.basename(pdf_path)}...")
logging.getLogger("pypdf").setLevel(logging.ERROR)
pdf_docs = []
pdf_loader_used = "PyPDFLoader"
try:
    if PyMuPDFLoader is not None:
        pdf_docs = PyMuPDFLoader(pdf_path).load()
        pdf_loader_used = "PyMuPDFLoader"
    else:
        raise ImportError("PyMuPDFLoader not available")
except Exception as e:
    print(f"   [PDF fallback] {e}")
    pdf_docs = PyPDFLoader(pdf_path).load()

clean_pdf_docs = []
for d in pdf_docs:
    text = (d.page_content or "").strip()
    if not text:
        continue
    d.page_content = "\n".join([line.strip() for line in text.splitlines() if line.strip()])
    d.metadata = {**d.metadata, "source": "project.pdf"}
    clean_pdf_docs.append(d)

pdf_docs = clean_pdf_docs
print(f"   PDF loaded with {pdf_loader_used}; pages kept: {len(pdf_docs)}")
# Pievienojam PDF lapas kopējam sarakstam
all_documents.extend(pdf_docs)


# --- B. ZINĀŠANU BĀZE (Vector Store) ---

print("   💾 Veidoju Vektoru Datubāzi (ChromaDB)...")

# Sadalām tekstu gabalos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(all_documents)

# Saglabājam ChromaDB (operatīvajā atmiņā)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

print(f"✅ Datu bāze gatava! Apstrādāti {len(splits)} informācijas gabali.")

🔎 Sāku datu apstrādi... (Tas var aizņemt pāris minūtes)
   🎤 Apstrādāju audio: meeting.mp3...
   🖼️ Apstrādāju attēlu: whiteboard.png...
   📄 Apstrādāju PDF: project.pdf...
   PDF loaded with PyMuPDFLoader; pages kept: 13
   💾 Veidoju Vektoru Datubāzi (ChromaDB)...
✅ Datu bāze gatava! Apstrādāti 40 informācijas gabali.


## Kods čata interfeisam (RAG funkcija)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# --- RAG SISTĒMAS UZBŪVE ---

# 1. Definējam LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 2. Definējam meklētāju
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 40, "lambda_mult": 0.15}
)


def list_sources():
    return sorted({d.metadata.get("source", "Nezināms") for d in splits if d.metadata.get("source")})


def dedupe_docs(docs):
    seen = set()
    unique_docs = []
    for d in docs:
        source = d.metadata.get("source", "Nezināms")
        page = d.metadata.get("page", None)
        key = (source, page, (d.page_content or "").strip()[:240])
        if key in seen:
            continue
        seen.add(key)
        unique_docs.append(d)
    return unique_docs


def retrieve_docs(question, per_source_k=3, global_k=4):
    combined_docs = []
    source_hits = {}

    # 1) Meklējam katra avota atsevišķi
    for source in list_sources():
        docs = []
        try:
            docs = vectorstore.similarity_search(
                question,
                k=per_source_k,
                filter={"source": source},
            )
        except Exception:
            docs = []

        if not docs:
            docs = [
                d for d in retriever.invoke(question)
                if d.metadata.get("source") == source
            ][:per_source_k]

        source_hits[source] = docs
        combined_docs.extend(docs)

    # 2) Pievienojam papildu globāli relevantos gabalus
    combined_docs.extend(retriever.invoke(question)[:global_k])

    return dedupe_docs(combined_docs), source_hits


def format_docs(docs):
    grouped = {}
    for d in docs:
        source = d.metadata.get("source", "Nezināms")
        grouped.setdefault(source, []).append(d)

    blocks = []
    for source in sorted(grouped.keys()):
        blocks.append(f"=== Avots: {source} ===")
        for i, d in enumerate(grouped[source], start=1):
            page = d.metadata.get("page")
            locator = f", lapa {page + 1}" if isinstance(page, int) else ""
            blocks.append(f"[{source}#{i}{locator}] {d.page_content}")
        blocks.append("")

    return "\n".join(blocks).strip()


def build_context(question):
    docs, source_hits = retrieve_docs(question)
    if not docs:
        return "NAV ATRASTS NEVIENS RELEVANTS FRAGMENTS."

    missing = [source for source, hits in source_hits.items() if not hits]
    header = ""
    if missing:
        header = (
            "PIEZĪME: šiem avotiem semantiskā meklēšana neatgrieza fragmentus: "
            + ", ".join(missing)
            + "\n\n"
        )

    return header + format_docs(docs)


# 3. Definējam promptu
template = """Tu esi uzņēmuma stratēģiskais AI asistents.
Tavs uzdevums ir atbildēt uz jautājumiem, izmantojot TIKAI zemāk sniegto kontekstu.

Noteikumi:
1) Izveido vienu apkopotu atbildi no visiem avotiem, kuros ir atbilstoša informācija.
2) Ja viens fakts redzams vairākos avotos, apvieno to viena teikumā un pievieno visus avotus.
3) Ja avoti savstarpēji pretrunojas, to arī norādi.
4) Pēc katra būtiska fakta norādi avotu iekavās (piem., [Avots: meeting.mp3], [Avots: project.pdf]).
5) Ja atbildes nav kontekstā, saki: "Diemžēl failos šādas informācijas nav."

Konteksts:
{context}

Jautājums: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 4. RAG ķēde
rag_chain = (
    {"context": RunnableLambda(build_context), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


# --- ČATA CILPA ---

print("Projekta Smadzenes ir gatavas. Jautā man jebko par failiem!")
print("Lai beigtu darbu, ieraksti: exit")

while True:
    # 1. Ievades lauks
    user_question = input("\nTavs Jautājums: ").strip()

    # 2. Izejas nosacījums
    if not user_question:
        continue
    if user_question.lower() in ["exit", "quit", "beigt", "-"]:
        print("Čats pabeigts.")
        break

    # 3. RAG izpilde
    try:
        print("   Analizēju datus...")
        response = rag_chain.invoke(user_question)

        # 4. Rezultāts
        retrieved_docs, _ = retrieve_docs(user_question)
        used_sources = sorted({d.metadata.get("source", "Nezināms") for d in retrieved_docs})

        print(f"\nJautājums:\n{user_question}")
        print(f"\nAtbilde:\n{response}")
        print(f"\nIzmantotie avoti: {', '.join(used_sources) if used_sources else 'Neviens'}")
        print("-" * 50)

    except Exception as e:
        print(f"Kļūda: {e}")


Projekta Smadzenes ir gatavas. Jautā man jebko par failiem!
Lai beigtu darbu, ieraksti: exit
   Analizēju datus...

Jautājums:
Ko sapulcē nolēma par budžetu?

Atbilde:
Sapulcē tika nolemts, ka pilnais apmaksātais mediju budžets novembrim netiks samazināts, pat ja palaišana notiks vēlāk. Tiks koncentrēta izdevumu veikšana mēneša otrajā pusē, lai pārbaudītu, vai AI ģenerētie raksti spēj piesaistīt klientus [Avots: meeting.mp3].

Izmantotie avoti: meeting.mp3, project.pdf, whiteboard.png
--------------------------------------------------
   Analizēju datus...

Jautājums:
Kādi skaitļi redzami uz tāfeles/rēķinā?

Atbilde:
Uz tāfeles redzami šādi skaitļi:

- $45,000
- $18,000
- $25,000
- $30,000
- $12,000
- $10,000
- $42,500
- $18,000
- $31,200
- $12,000
- $1,500
- $140,000
- ~$105,200
- Oct 15

[Avots: whiteboard.png]

Izmantotie avoti: meeting.mp3, project.pdf, whiteboard.png
--------------------------------------------------
